In [ ]:
from os import path
from PIL import Image
from colorthief import ColorThief
import matplotlib.pyplot as plt
import numpy as np

DATAPATH = './nolabels'

image_paths = [
    ('kishan_bagh_nolabels.png', 19.23), # 2 green shades
    ('firewood_consumption_nolabels.png', 8.038), # 1 shade works
    ('rao_jodha1_nolabels.png', 16.58), # 1 shade works
    ('auroraville_botanical_gardens_nolabels.png', 78.63), # Prefer only second
    ('la_joya_nolabels.png', 92.01), # 1 green shade
    ('jussara_nolabels.png', 85.83) # 2 green shades
]

DEFORESTATION_PATH = './deforestation'

deforestation_paths = [
    'deforestation_0.png',
    'deforestation_1.png',
    'deforestation_2.png'
]
     
PALETTE_SIZE = 6
MOST_GREEN = 2

In [ ]:
def greenness_rgb(color):
    r, g, b = color
    #return g/max(1,b) + g/max(1,r) # custom greenness
    return -np.linalg.norm(np.array((0, 255, 0)) - np.array(color)) # DtP greenness

greenness_rgb((33, 59, 27))

def percentage_crop(percentage):
    string = str(percentage)

    if len(string) < 6:
        return string
    else:
        return string[:5]
    

In [ ]:
def tree_coverage(image_path: str):
    image = Image.open(image_path)
    ct = ColorThief(image_path)
    palette = ct.get_palette(color_count=PALETTE_SIZE)
    palette.sort(key = lambda x: greenness_rgb(x), reverse = True)

    # This dict will be used to determine which of the MOST_GREEN colors will be used
    used_colors = {i: True for i in range(MOST_GREEN)}
    
    for i in range(MOST_GREEN):
        r, g, b = palette[i]
        if r > g or b > g:
            used_colors[i] = False
    
    print(palette)
    
    image_copy = image.copy()
    image_size = image.width * image.height

    for row_nr in range(image_copy.height):
        for col_nr in range(image_copy.width):
            position = (col_nr, row_nr)
            pixel = image.getpixel(position)

            norm_func = lambda color: np.linalg.norm(np.array(color) - np.array(pixel))
            best_fit = tuple(palette[np.argmin(np.array(list(map(norm_func, palette))))])

            image_copy.putpixel(position, best_fit)


    image_copy.save(image_path.split('.')[0] + "new.png")
    image_to_list = list(image_copy.getdata())

    #display(image_copy)
    
    image_to_list = list(image_copy.getdata())
    percentage_list = []

    for i in range(MOST_GREEN):
        if used_colors[i]:
            percentage_list.append(100 * image_to_list.count(palette[i]) / image_size)

            print(f'Percentage of palette[{i}]: {percentage_crop(percentage_list[-1])}%')

    

    print(f'Total coverage: {percentage_crop(sum(percentage_list))}%')
    return sum(percentage_list)

In [ ]:
# TEST

for file, correct_coverage in image_paths:
    coverage = tree_coverage(path.join(DATAPATH, file))
    if abs(coverage - correct_coverage) < 4:
        print(f'{file}: Measured correctly')
    else:
        print(f'{file}: Measured incorrectly')

In [ ]:
# 5 colors, 2 greens, custom greenness and no throwaway: 3/6
# 6 colors, 2 greens, custom greenness and no throwaway: 5/6
# 5 colors, 2 greens, custom greenness and throwaway: 4/6
# 6 colors, 2 greens, custom greenness and throwaway: 4/6

# 5 colors, 2 greens, DtP greenness and no throwaway: 3/6
# 6 colors, 2 greens, DtP greenness and no throwaway: 3/6
# 5 colors, 2 greens, DtP greenness and throwaway: 4/6
# 6 colors, 2 greens, DtP greenness and throwaway: 3/6

In [ ]:
deforestation_coverages = [tree_coverage(path.join(DEFORESTATION_PATH, file)) for file in deforestation_paths]

In [ ]:
print(deforestation_coverages)